In [10]:
import pandas as pd
import numpy as np
import datetime
import math
import time

from numpy import array
from pandas import DataFrame, read_csv, concat
from keras import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler


In [16]:
df = pd.read_csv('datapoints.csv')
df['date'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S')
df = df.set_index(pd.DatetimeIndex(df['date']))
df.drop(['attribute_name', 'date'], axis=1, inplace=True)
df["datetime"] = df.index 

mask = df.name.str.contains("BLOK61")
df_blok = df[mask]
df_videolab = df[~mask]
df_videolab_filtered = df_videolab["value"].to_numpy()

df_videolab


In [12]:
df_videolab = df_videolab.resample(rule='15T', closed='left', label='left').mean().apply(np.ceil)


In [13]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
	# input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
	# drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

df_videolab = series_to_supervised(df_videolab, n_out=4)

df_videolab


,var1(t-1),var1(t),var1(t+1),var1(t+2),var1(t+3)
date,,,,,
2021-04-05 02:15:00,3.0,3.0,3.0,3.0,3.0
2021-04-05 02:30:00,3.0,3.0,3.0,3.0,3.0
2021-04-05 02:45:00,3.0,3.0,3.0,3.0,3.0
2021-04-05 03:00:00,3.0,3.0,3.0,3.0,3.0
2021-04-05 03:15:00,3.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...
2021-05-06 14:45:00,11.0,11.0,11.0,10.0,8.0
2021-05-06 15:00:00,11.0,11.0,10.0,8.0,7.0
2021-05-06 15:15:00,11.0,10.0,8.0,7.0,7.0


In [14]:
values = df_videolab.values

In [15]:
...
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()



NameError: name 'train_X' is not defined

In [66]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

UnboundLocalError: local variable 'batch_outputs' referenced before assignment